In [1]:
import s3fs
import xarray as xr

In [42]:
# 1. Point s3fs at the public bucket (no credentials needed)
fs = s3fs.S3FileSystem(anon=True)
date = 20250603
# 2. List a directory (e.g. daily time_series files for soil moisture)
path = f"noaa-gfs-bdp-pds/gdas.{date}/00/atmos/"  # adjust to your year/month dynamically
file = fs.glob(
    path + "gdas.t00z.sfluxgrbf000.grib2"
)  

grib_file = "data/gdas.t00z.sfluxgrbf000.grib2"
# 3. Open as a single concatenated Dataset
ds_surface = xr.open_dataset(
    grib_file,
    engine="cfgrib",
    filter_by_keys={'typeOfLevel': 'surface'},
)

grib_file = "data/gdas.t00z.sfluxgrbf000.grib2"
# 3. Open as a single concatenated Dataset
ds_below = xr.open_dataset(
    grib_file,
    engine="cfgrib",
    filter_by_keys={'typeOfLevel': 'depthBelowLandLayer', 'level': 0.0},
)



/Users/CraigC/Documents/Learning/Mapping_Coding/Projects/can_i_ride_research/.venv/lib/python3.13/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
skipping variable: paramId==228039 shortName='sm'
Traceback (most recent call last):
  File "/Users/CraigC/Documents/Learning/Mapping_Coding/Projects/can_i_ride_research/.venv/lib/python3.13/site-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
    ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/CraigC/Documents/Learning/Mapping_Coding/Projects/can_i_ride_research/.venv/lib/python3.13/site-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
    ...<2 lines>...
    )
cfgrib.dataset.DatasetBuildError: key 

In [50]:
ds_surface_us = ds_surface.sel(longitude=slice(-126.59,-65.6), latitude=slice(24.4,50.59))
ds_surface_us

<xarray.Dataset> Size: 32B
Dimensions:     (latitude: 0, longitude: 0)
Coordinates:
    time        datetime64[ns] 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
  * latitude    (latitude) float64 0B 
  * longitude   (longitude) float64 0B 
    valid_time  datetime64[ns] 8B ...
Data variables: (12/29)
    sp          (latitude, longitude) float32 0B ...
    orog        (latitude, longitude) float32 0B ...
    t           (latitude, longitude) float32 0B ...
    cnwat       (latitude, longitude) float32 0B ...
    sdwe        (latitude, longitude) float32 0B ...
    sde         (latitude, longitude) float32 0B ...
    ...          ...
    sdswrf      (latitude, longitude) float32 0B ...
    sdlwrf      (latitude, longitude) float32 0B ...
    suswrf      (latitude, longitude) float32 0B ...
    sulwrf      (latitude, longitude) float32 0B ...
    lsm         (latitude, longitude) float32 0B ...
    siconc      (latitude, longitude) float32 0B ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2025-06-03T20:49 GRIB to CDM+CF via cfgrib-0.9.1...

In [47]:
ds_surface[['wilt','fldcp','SUNSD', 'sltyp']]

<xarray.Dataset> Size: 76MB
Dimensions:     (latitude: 1536, longitude: 3072)
Coordinates:
    time        datetime64[ns] 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
  * latitude    (latitude) float64 12kB 89.91 89.79 89.68 ... -89.79 -89.91
  * longitude   (longitude) float64 25kB 0.0 0.1172 0.2344 ... 359.6 359.8 359.9
    valid_time  datetime64[ns] 8B ...
Data variables:
    wilt        (latitude, longitude) float32 19MB ...
    fldcp       (latitude, longitude) float32 19MB ...
    SUNSD       (latitude, longitude) float32 19MB ...
    sltyp       (latitude, longitude) float32 19MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2025-06-03T20:49 GRIB to CDM+CF via cfgrib-0.9.1...

In [44]:
ds_below['soilw']

<xarray.DataArray 'soilw' (depthBelowLandLayer: 3, latitude: 1536,
                           longitude: 3072)> Size: 57MB
[14155776 values with dtype=float32]
Coordinates:
    time                 datetime64[ns] 8B ...
    step                 timedelta64[ns] 8B ...
  * depthBelowLandLayer  (depthBelowLandLayer) float64 24B 0.0 0.1 0.4
  * latitude             (latitude) float64 12kB 89.91 89.79 ... -89.79 -89.91
  * longitude            (longitude) float64 25kB 0.0 0.1172 ... 359.8 359.9
    valid_time           datetime64[ns] 8B ...
Attributes: (12/30)
    GRIB_paramId:                             260185
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      4718592
    GRIB_typeOfLevel:                         depthBelowLandLayer
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                Volumetric soil moisture content
    GRIB_shortName:                           soilw
    GRIB_units:                               Proportion
    long_name:                                Volumetric soil moisture content
    units:                                    Proportion
    standard_name:                            unknown